In [1]:
import pandas as pd, numpy as np, json, re, pickle, keras

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
import string
#import Keras libraries to build a neural network classifier
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
def read_data(file):
    """
    Take a json file location and
    read the file into a pandas data frame
    Args: full path to file
    Returns: pandas dataframe with data from file
    """
    
    data = []

    with open(file) as f:
        for line in f:
            data.append(json.loads(line))
        
    # convert to data frame
    
    return pd.DataFrame(data)

In [3]:
# read category data
# Had to complete the missing " and } in the file. Was getting error earlier.
df = read_data('categorized-comments.jsonl')

# check size, structure and categories

print('Size: ', len(df), '\n',
      'Shape: ', df.info(), '\n',
      'Categories: ', df.cat.unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606474 entries, 0 to 606473
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   cat     606474 non-null  object
 1   txt     606474 non-null  object
dtypes: object(2)
memory usage: 9.3+ MB
Size:  606474 
 Shape:  None 
 Categories:  ['sports' 'science_and_technology' 'video_games']


In [4]:
df = df.sample(n=5000)
df["cat"].value_counts()

video_games               3596
sports                    1199
science_and_technology     205
Name: cat, dtype: int64

In [5]:
df.apply(lambda x: x.astype(str).str.lower())

,cat,txt
369915,sports,unicorn and willy\n\nhttps://www.youtube.com/w...
143364,video_games,[removed]
229003,sports,it doesn't matter one bit. if you want aiming ...
123062,video_games,lol wouldnt be surprised at this point
310272,video_games,"as an actual lawyer, i'm finding this thread t..."
...,...,...
264311,sports,same with fut draft...
179340,video_games,yeah it's almost like the bottle is available ...
152732,video_games,"oh hey there lunatic, it almost seems like the..."
317241,video_games,the singleplayer is fantastic. zombies co-op i...


In [6]:
#remove punctuation
df['txt'] = df['txt'].apply(lambda x:''.join([i for i in x 
                                                  if i not in string.punctuation]))

In [7]:
stop = stopwords.words('english')

In [8]:
import nltk
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
df['txt'] = df['txt'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

In [10]:
print(df['txt'])

369915     Unicorn Willy httpswwwyoutubecomwatchvJBJZdnwp1w
143364                                              removed
229003    It doesnt matter one bit If want aiming matter...
123062                          LOL wouldnt surprised point
310272    As actual lawyer Im finding thread bit cringe ...
                                ...                        
264311                                            FUT draft
179340    yeah almost like bottle available side shop sh...
152732    Oh hey Lunatic almost seems like dont want gam...
317241    The Singleplayer fantastic Zombies CoOp lot fu...
213854    Yeah I get If ever want try Brozo I recommend ...
Name: txt, Length: 5000, dtype: object


In [11]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
porter_stemmer = PorterStemmer()
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['txt'] = df['txt'].apply(stem_sentences)

In [12]:
# set the features and classes

N_FEATURES = 3000
N_CLASSES = 3
N_UNITS = 2500

In [13]:
# create the feature matrix
#do tfidf
cv = TfidfVectorizer(analyzer='word',
                     stop_words=stop, 
                     max_features = N_FEATURES,
                     max_df = 0.5,
                     min_df = 3)

# create target and sample

X = cv.fit_transform(df['txt'])
y = df['cat']

# create train test split
# Splitting 75 25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [14]:
#A simple ANN can only take a linear array of features as input. Therefore, checking the train and test dataset accordingly.
print(str(X_train.shape))
print(str(X_test.shape))
print(str(y_train.shape))
print(str(y_test.shape))

(3750, 3000)
(1250, 3000)
(3750,)
(1250,)


In [15]:
print(y_train)

2818           sports
191915    video_games
151549    video_games
556716    video_games
274280    video_games
             ...     
255108         sports
357871    video_games
276277    video_games
209745         sports
227160         sports
Name: cat, Length: 3750, dtype: object


In [16]:
# initialize

classifier_seq = Sequential()

classifier_seq.add(Dense(units=500,activation="relu",input_shape=(N_FEATURES,)))
classifier_seq.add(Dense(units=50, activation="relu"))
classifier_seq.add(Dense(units=N_CLASSES, activation="softmax"))

# compile the Artificial Neural Network (ANN)

classifier_seq.compile(optimizer="adam", 
                       loss="categorical_crossentropy", 
                       metrics=["accuracy"])

In [17]:
#build the model architecture
def build_network():
    nn = Sequential()
    nn.add(Dense(500, activation = 'relu', input_shape = (N_FEATURES,)))
    nn.add(Dense(150, activation = 'relu'))
    nn.add(Dense(N_CLASSES, activation = 'softmax'))
    nn.compile(
        loss = 'categorical_crossentropy',
        optimizer = 'adam',
        metrics = ['accuracy']
    )
    return nn

In [18]:
print(str(X_train.shape))
print(str(X_test.shape))
print(str(y_train.shape))
print(str(y_test.shape))

(3750, 3000)
(1250, 3000)
(3750,)
(1250,)


In [19]:
X_train = X_train.todense()

In [20]:
#train the model
nn_mod2 = KerasClassifier(build_fn = build_network, epochs = 200, batch_size = 128)
#nn_mod2.fit(X_train, y_train, validation_data = (X_val, y_val))
nn_mod2.fit(X_train, y_train)

Epoch 1/200
30/30 [==============================] - 2s 25ms/step - loss: 0.9119 - accuracy: 0.6623
Epoch 2/200
30/30 [==============================] - 1s 27ms/step - loss: 0.5958 - accuracy: 0.7288
Epoch 3/200
30/30 [==============================] - 1s 26ms/step - loss: 0.4301 - accuracy: 0.8033
Epoch 4/200
30/30 [==============================] - 1s 26ms/step - loss: 0.2713 - accuracy: 0.8949
Epoch 5/200
30/30 [==============================] - 1s 27ms/step - loss: 0.1958 - accuracy: 0.9096
Epoch 6/200
30/30 [==============================] - 1s 24ms/step - loss: 0.1372 - accuracy: 0.9368
Epoch 7/200
30/30 [==============================] - 1s 20ms/step - loss: 0.1186 - accuracy: 0.9612
Epoch 8/200
30/30 [==============================] - 1s 23ms/step - loss: 0.0947 - accuracy: 0.9661
Epoch 9/200
30/30 [==============================] - 1s 22ms/step - loss: 0.0871 - accuracy: 0.9655
Epoch 10/200
30/30 [==============================] - 1s 21ms/step - loss: 0.0767 - accuracy: 0.9668

In [21]:
#eval and calc matrix
classifier_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               1500500   
_________________________________________________________________
dense_1 (Dense)              (None, 50)                25050     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 153       
Total params: 1,525,703
Trainable params: 1,525,703
Non-trainable params: 0
_________________________________________________________________
